The master_file serves as the glue for all other files in the project.  It is the central file to load training and test data, tag the addresses, standardize the addresses, and compare the different address lists.

In [29]:
from collections import defaultdict
from address_compare import standardizers as stndrdzr
from address_compare import comparers as comps
from address_compare import matcher as mtch
from address_compare.crf_tagger import AddressTagger
import json
import pandas as pd

In [30]:
# Editable parameters to control the functions
retrain_crf_tagger = False
standardize_addresses = True

file_location_raw_addresses_1 = 'address_compare\\data\\sandbox data.xlsx'
raw_address_field_name_list_1 = 'Single String Address'

file_location_raw_addresses_2 = 'address_compare\\data\\sandbox data.xlsx'
raw_address_field_name_list_2 = 'Single String Address'

In [31]:
# Placeholder for reading/calling the training data for the CRF Tagger and sending the training data to train the model
if retrain_crf_tagger:
    with open('data/tagged_addresses.json') as f:
        td = json.load(f)
    
    #send training data to CRF tagger to train the model here...

In [32]:
# Placeholder for reading/calling the 2 lists of raw addresses
raw_address_list_1 = pd.read_excel(file_location_raw_addresses_1)
raw_address_list_2 = pd.read_excel(file_location_raw_addresses_2)

In [33]:
# Create List for Raw Address Dataframes
raw_addresses = [raw_address_list_1, raw_address_list_2]

In [34]:
# Add a field called Record_ID if it doesn't already exist in the raw address files
for dtfrm in raw_addresses:
    dtfrm = stndrdzr.record_id_addition(dtfrm)

In [35]:
# Add Empty Missing Columns to Dataframe
missing_columns = ['CITY','STATE','ZIP_CODE','UNKNOWN']
for dtfrm in raw_addresses:
    dtfrm = stndrdzr.empty_column_addition(dtfrm, missing_columns)

In [36]:
# instantiate AddressTagger object with default options, which gives the model trained in `Train CRF Model`.ipynb
at = AddressTagger()

In [37]:
# Placeholder for calling the trained CRF Tagger on the 2 lists of raw addresses
tagged_address_list_1 = at.series_to_address_df(raw_address_list_1[raw_address_field_name_list_1], standardize = standardize_addresses)
tagged_address_list_2 = at.series_to_address_df(raw_address_list_2[raw_address_field_name_list_2], standardize = standardize_addresses)

In [38]:
# Add Remaining Columns from Raw Address Dataframes to Tagged Address Dataframes
joined_address_list_1 = tagged_address_list_1.join(raw_address_list_1)
joined_address_list_2 = tagged_address_list_2.join(raw_address_list_2)

In [39]:
print (joined_address_list_1)

   STREET_NUMBER PRE_DIRECTION       STREET_NAME STREET_TYPE POST_DIRECTION  \
0           1732                        BROADWAY                              
1            747                        BROADWAY                              
2            433                            10TH      AVENUE                  
3           1420                            10TH      AVENUE                  
4            210                            10TH      STREET                  
5            137                            11TH      AVENUE                  
6            167                            11TH      AVENUE                  
7            319                            11TH       PLACE                  
8            129                            12TH      AVENUE                  
9              3                BRITISH PROPERTY         WAY                  
10          1732                        BROADWAY                              
11           747                        BROADWAY    

In [40]:
print (joined_address_list_2)

   STREET_NUMBER PRE_DIRECTION       STREET_NAME STREET_TYPE POST_DIRECTION  \
0           1732                        BROADWAY                              
1            747                        BROADWAY                              
2            433                            10TH      AVENUE                  
3           1420                            10TH      AVENUE                  
4            210                            10TH      STREET                  
5            137                            11TH      AVENUE                  
6            167                            11TH      AVENUE                  
7            319                            11TH       PLACE                  
8            129                            12TH      AVENUE                  
9              3                BRITISH PROPERTY         WAY                  
10          1732                        BROADWAY                              
11           747                        BROADWAY    

In [41]:
# Intra-Grouping of Tagged Address Lists to Consolidate Duplicates
grouped_address_list_1 = stndrdzr.consolidate_address_list(joined_address_list_1)
grouped_address_list_2 = stndrdzr.consolidate_address_list(joined_address_list_2)

In [42]:
# Call Either the Exact Match or Learning Match Functions to match the 2 lists
